# I. Librerías usadas

Aquí se reunen todas las importaciones usadas para la carga del corpus, análisis y tokenización de términos, limpieza de términos y el modelo en general.

In [ ]:
# Estas librerías tratan con los datos extraídos de la base de datos
import pandas as pd
import numpy as np

# Estas librerías son utilizadas para el preprocesamiento de palabras o términos y tokenización
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Esta librería nos va a servir para serializar objetos, en nuestro caso, los modelos creados.
# Se serializan en bytes en un archivo, y se pueden volver a cargar en otro código
import pickle

# Simplemente para tratar con los tipos de datos de listas en python
from typing import List

# II. Modelos de búsqueda

Este proyecto hará uso de 3 modelos diferentes de búsqueda, para así poder analizar cuál es el mejor.

## 1. Modelo Binario

Para este caso, utilizamos clases que nos permiten interactuar fácilmente con los métodos y atributos del modelo.

In [ ]:
class ModeloBinario:
    """
    Utiliza una matriz de ocurrencia término-documento.
    """
    def __init__(self):
        self.vocabulario = {} # Diccionario de término a ID
        self.matrizOcurrencia = None # Matriz de NumPy (Documentos x Términos)
        self.listaDocumentos = [] # Lista de IDs/Índices de documentos
        self.listaStopwords = set(stopwords.words('english'))

    def preProcesar(self, texto):
        """ Tokenización y eliminación de stopwords para un texto. """
        # Convertir a minúsculas
        textoMin = texto.lower()
        # Tokenizar (NLTK es permitido)
        tokens = word_tokenize(textoMin)
        # Filtrar stopwords y tokens no alfabéticos
        tokensFiltrados = [
            token for token in tokens
            if token.isalpha() and token not in self.listaStopwords
        ]
        return tokensFiltrados

    def ajustarCorpus(self, serieDocumentos):
        """
        'Ajusta' el modelo al corpus, creando el vocabulario y la matriz.
        serieDocumentos debe ser la columna 'Answer' del DataFrame.
        """
        print("\nIniciando ajuste del Modelo Binario...")
        documentosTokenizados = []
        documentoID = 0

        # 1. Generar tokens y vocabulario
        for texto in serieDocumentos:
            tokens = self.preProcesar(texto)
            documentosTokenizados.append(tokens)
            self.listaDocumentos.append(documentoID) # Usamos el índice de la serie como ID
            documentoID += 1

            for token in tokens:
                if token not in self.vocabulario:
                    # Asignar un ID único a cada término
                    self.vocabulario[token] = len(self.vocabulario)

        # 2. Crear Matriz de Ocurrencia (Documentos x Términos)
        numDocs = len(documentosTokenizados)
        numTerminos = len(self.vocabulario)

        # Inicializar la matriz con ceros
        self.matrizOcurrencia = np.zeros((numDocs, numTerminos), dtype=np.int8)

        # Llenar la matriz con 1s para indicar presencia
        for docIndex, tokens in enumerate(documentosTokenizados):
            for token in tokens:
                # Obtener el ID del término
                terminoIndex = self.vocabulario[token]
                # Marcar presencia (1)
                self.matrizOcurrencia[docIndex, terminoIndex] = 1

        print(f"Ajuste completado. Documentos: {numDocs}, Términos: {numTerminos}")
        print("Matriz de Ocurrencia (Documentos x Términos):")
        print(self.matrizOcurrencia)

    def buscar(self, consulta, k=3):
        """
        Realiza una búsqueda simple (AND) y devuelve los primeros k resultados.
        """
        print(f"\nBuscando: '{consulta}' con límite k={k}")
        tokensConsulta = self.preProcesar(consulta)

        # Máscara binaria para la relevancia: todos los documentos inicialmente relevantes
        relevanciaBooleana = np.ones(len(self.listaDocumentos), dtype=bool)

        terminosNoEncontrados = []

        # 1. Aplicar la lógica AND para cada término de la consulta
        for token in tokensConsulta:
            if token in self.vocabulario:
                terminoIndex = self.vocabulario[token]
                vectorTermino = self.matrizOcurrencia[:, terminoIndex]

                # Operación AND
                relevanciaBooleana = relevanciaBooleana & (vectorTermino == 1)
            else:
                terminosNoEncontrados.append(token)
                relevanciaBooleana[:] = False
                break

        # 2. Obtener los índices de los documentos relevantes
        indicesRelevantes = np.where(relevanciaBooleana)[0]

        # 3. Aplicar el límite k
        indicesLimitados = indicesRelevantes[:k]

        if len(indicesRelevantes) == 0:
            print("No se encontraron documentos relevantes.")
            return []

        print(f"Documentos relevantes encontrados (total): {len(indicesRelevantes)}")
        print(f"Documentos retornados (top k={k}): {len(indicesLimitados)}")
        print(f"Top {k} resultados encontrados (ID):")
        return indicesLimitados

## Modelo Vectorial (TF-IDF)

En el modelo vectorial, nos enfocamos en encapsular los métodos en una clase para organizar los diferentes métodos para una búsqueda.

In [ ]:
class ModeloVectorialTfIdf:
    """
    Implementación del Modelo Vectorial utilizando la ponderación TF-IDF.
    """
    def __init__(self):
        self.vocabulario = {}        # Término a ID (índice de columna)
        self.vectorIdf = None        # Vector de NumPy con los pesos IDF
        self.matrizTfIdf = None       # Matriz de NumPy (Documentos x Términos)
        self.listaStopwords = set(stopwords.words("english"))
        self.listaDocumentos = []    # Lista de IDs/Índices de documentos
        self.numDocumentos = 0       # Total de documentos en el corpus

    def preProcesar(self, texto):
        """ Tokenización y eliminación de stopwords (reutilizado). """
        textoMin = texto.lower()
        tokens = word_tokenize(textoMin)
        tokensFiltrados = [
            token for token in tokens
            if token.isalpha() and token not in self.listaStopwords
        ]
        return tokensFiltrados

    # --- Ponderación del Modelo ---

    def calcularTf(self, docTokens):
        """ Calcula la Frecuencia de Término (TF) para un documento. """
        frecuencias = {}
        for token in docTokens:
            frecuencias[token] = frecuencias.get(token, 0) + 1

        # normalización (por longitud del documento)
        # longitud = len(docTokens)
        # tfVector = np.array([frecuencias.get(t, 0) / longitud for t in self.vocabulario.keys()])
        return frecuencias

    def calcularIdf(self, matrizTf):
        """ Calcula la Frecuencia Inversa de Documento (IDF) para todos los términos. """
        # Frecuencia de documento (df): cuántos documentos contienen el término
        documentosConTermino = np.sum(matrizTf > 0, axis=0)

        # division por cero
        # log(N / df_t) + 1
        idfVector = np.log((self.numDocumentos + 1) / (documentosConTermino + 1)) + 1

        return idfVector

    def normalizarMatriz(self, matriz):
        """ Normaliza los vectores de la matriz a longitud unitaria (norma L2). """
        # Calcular la norma euclidiana (L2-norm) de cada fila (vector de documento)
        normas = np.linalg.norm(matriz, axis=1)
        matrizNormalizada = np.divide(
            matriz,
            normas[:, np.newaxis],
            out=np.zeros_like(matriz, dtype=float), # Si la norma es 0, deja el vector como 0
            where=normas[:, np.newaxis]!=0
        )
        return matrizNormalizada

    def ajustarCorpus(self, serieDocumentos):
        """
        Crea el vocabulario, la matriz de frecuencia y calcula la matriz TF-IDF final.
        """

        documentosTokenizados = []
        for docId, texto in enumerate(serieDocumentos):
            tokens = self.preProcesar(texto)
            documentosTokenizados.append(tokens)
            self.listaDocumentos.append(docId)
            for token in tokens:
                if token not in self.vocabulario:
                    self.vocabulario[token] = len(self.vocabulario)

        self.numDocumentos = len(self.listaDocumentos)
        numTerminos = len(self.vocabulario)

        # matriz de Frecuencia de Término (Count Matrix)
        matrizFrecuencia = np.zeros((self.numDocumentos, numTerminos), dtype=np.int32)
        for docIndex, tokens in enumerate(documentosTokenizados):
            frecuencias = self.calcularTf(tokens)
            for token, freq in frecuencias.items():
                if token in self.vocabulario:
                    terminoIndex = self.vocabulario[token]
                    matrizFrecuencia[docIndex, terminoIndex] = freq

        # calcular IDF
        self.vectorIdf = self.calcularIdf(matrizFrecuencia)

        # calcular Matriz TF-IDF (Term Frequency * Inverse Document Frequency)
        # multiplicación elemento a elemento de la matriz TF por el vector IDF (broadcasting)
        matrizTfIdfCruda = matrizFrecuencia * self.vectorIdf

        # normalizar la Matriz TF-IDF
        self.matrizTfIdf = self.normalizarMatriz(matrizTfIdfCruda)

        print(f"Ajuste completado. Documentos: {self.numDocumentos}, Términos: {numTerminos}")
        print("Muestra de la Matriz TF-IDF (Normalizada):")
        print(self.matrizTfIdf)

    # --- Búsqueda (Search) del Modelo ---

    def buscar(self, consulta, k=3):
        """
        Calcula la similitud de la consulta con todos los documentos (Similitud del Coseno)
        y devuelve los 'k' documentos más relevantes.
        """
        tokensConsulta = self.preProcesar(consulta)

        # 1. Convertir la consulta a un vector TF-IDF
        vectorConsulta = np.zeros(len(self.vocabulario), dtype=float)

        # Calcular TF de la consulta
        frecuenciasConsulta = self.calcularTf(tokensConsulta)

        for token, freq in frecuenciasConsulta.items():
            if token in self.vocabulario:
                terminoIndex = self.vocabulario[token]
                # Ponderación TF-IDF: TF de la consulta * IDF del corpus
                vectorConsulta[terminoIndex] = freq * self.vectorIdf[terminoIndex]

        # 2. Normalizar el vector de consulta
        # La norma del vector de consulta
        normaConsulta = np.linalg.norm(vectorConsulta)
        if normaConsulta > 0:
            vectorConsultaNormalizado = vectorConsulta / normaConsulta
        else:
            return []

        # 3. Calcular Similitud del Coseno
        # Similitud del Coseno = A . B / (||A|| * ||B||)
        # Como ambos (matrizTfIdf y vectorConsultaNormalizado) ya están normalizados (norma 1),
        # la Similitud del Coseno es simplemente el producto punto:
        # Cos(theta) = MatrizTfIdf . VectorConsultaNormalizado_transpuesto

        # Producto punto entre la matriz (D x T) y el vector (T)
        similitudes = self.matrizTfIdf @ vectorConsultaNormalizado.T

        # 4. Obtener los índices de los documentos ordenados por similitud (descendente)
        # np.argsort devuelve los índices que ordenarían el array
        indicesOrdenados = np.argsort(similitudes)[::-1]

        # Obtener las puntuaciones de los documentos relevantes (top K)
        topKIndices = indicesOrdenados[:k]
        topKScores = similitudes[topKIndices]

        resultados = [(self.listaDocumentos[i], topKScores[idx]) for idx, i in enumerate(topKIndices)]

        print(f"Top {k} resultados encontrados (ID, Similitud del Coseno):")
        return resultados

## Modelo Probabilístico (BM25)

In [ ]:
class ModeloBM25:
    """
    Implementación del Modelo de Ranking BM25.
    """
    def __init__(self, k1=1.2, b=0.75, idioma='spanish'):
        self.k1 = k1                       # Parámetro de ajuste de saturación de TF
        self.b = b                         # Parámetro de ajuste de normalización por longitud
        self.vocabulario = {}              # Término a ID (índice de columna)
        self.listaStopwords = set(stopwords.words(idioma))
        self.listaDocumentos = []          # Lista de IDs/Índices de documentos
        self.matrizFrecuencia = None       # Matriz de NumPy (Documentos x Términos)
        self.vectorLongitudDocumento = None# Vector con la longitud de cada documento |D|
        self.longitudPromedio = 0.0        # Longitud promedio de los documentos avgdl
        self.vectorIdf = None              # Vector de NumPy con los pesos IDF de BM25

    def preProcesar(self, texto):
        """ Tokenización y eliminación de stopwords. """
        textoMin = texto.lower()
        tokens = word_tokenize(textoMin)
        tokensFiltrados = [
            token for token in tokens
            if token.isalpha() and token not in self.listaStopwords
        ]
        return tokensFiltrados

    # --- Ajuste (Fit) del Modelo ---

    def ajustarCorpus(self, serieDocumentos):
        """
        Crea el vocabulario, calcula las longitudes de documento, IDF,
        y la matriz de frecuencia necesaria para la puntuación.
        """
        documentosTokenizados = []
        longitudes = []

        # 1. Tokenizar, generar vocabulario y calcular longitudes
        for docId, texto in enumerate(serieDocumentos):
            tokens = self.preProcesar(texto)
            documentosTokenizados.append(tokens)
            longitudes.append(len(tokens))
            self.listaDocumentos.append(docId)
            for token in tokens:
                if token not in self.vocabulario:
                    self.vocabulario[token] = len(self.vocabulario)

        self.vectorLongitudDocumento = np.array(longitudes, dtype=float)
        self.numDocumentos = len(self.listaDocumentos)
        self.longitudPromedio = np.mean(self.vectorLongitudDocumento)
        numTerminos = len(self.vocabulario)

        # 2. Crear Matriz de Frecuencia de Término (Count Matrix)
        self.matrizFrecuencia = np.zeros((self.numDocumentos, numTerminos), dtype=np.int32)
        for docIndex, tokens in enumerate(documentosTokenizados):
            for token in tokens:
                if token in self.vocabulario:
                    terminoIndex = self.vocabulario[token]
                    self.matrizFrecuencia[docIndex, terminoIndex] += 1

        # 3. Calcular IDF (Específico de BM25)
        # BM25 IDF: log( (N - df_t + 0.5) / (df_t + 0.5) )
        documentosConTermino = np.sum(self.matrizFrecuencia > 0, axis=0) # df_t
        N = self.numDocumentos

        # Uso de NumPy para aplicar la fórmula a todos los términos
        self.vectorIdf = np.log((N - documentosConTermino + 0.5) / (documentosConTermino + 0.5))

        print(f"Ajuste completado. Documentos: {self.numDocumentos}, Términos: {numTerminos}")
        print(f"Longitud Promedio (avgdl): {self.longitudPromedio:.2f}")


    # --- Búsqueda (Search) del Modelo ---

    def buscar(self, consulta, k=3):
        """
        Calcula las puntuaciones BM25 para la consulta y ranquea los documentos.
        """
        tokensConsulta = self.preProcesar(consulta)
        puntuaciones = np.zeros(self.numDocumentos, dtype=float)

        # Normalización por longitud (B): k1 * (1 - b + b * (|D| / avgdl))
        normalizacionDoc = self.k1 * (
            (1 - self.b) + self.b * (self.vectorLongitudDocumento / self.longitudPromedio)
        )

        for token in tokensConsulta:
            if token in self.vocabulario:
                terminoIndex = self.vocabulario[token]
                # Obtener la columna de IDF y la columna de frecuencia (tf)
                idfTermino = self.vectorIdf[terminoIndex]
                frecuenciasTermino = self.matrizFrecuencia[:, terminoIndex] # f(t_i, D)

                # Expresión del numerador de la fórmula: f(t_i, D) * (k1 + 1)
                numerador = frecuenciasTermino * (self.k1 + 1)

                # Expresión del denominador: f(t_i, D) + Normalización por longitud
                denominador = frecuenciasTermino + normalizacionDoc

                # Acumular puntuaciones (IDF * Factor de saturación/normalización)
                if idfTermino > 0:
                    puntuaciones += idfTermino * (numerador / denominador)

        # 1. Obtener los índices de los documentos ordenados por puntuación (descendente)
        indicesOrdenados = np.argsort(puntuaciones)[::-1]

        # 2. Seleccionar los top K documentos con puntuaciones > 0
        topKIndices = [i for i in indicesOrdenados if puntuaciones[i] > 0][:k]
        topKScores = puntuaciones[topKIndices]

        if len(topKIndices) == 0:
            print("No se encontraron documentos relevantes.")
            return []

        resultados = [(self.listaDocumentos[i], topKScores[idx]) for idx, i in enumerate(topKIndices)]

        print(f"Top {k} resultados encontrados (ID, Puntuación BM25):")
        return resultados

# III. Preprocesamiento del Corpus

## 1. Importación de los documentos para generar el Corpus

La información está esparcida en varios documentos, así que aquí los unificamos en un solo dataframe.

In [ ]:
archivos = [
    "/content/CancerQA.csv",
    "/content/Genetic_and_Rare_DiseasesQA.csv",
    "Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv",
    "SeniorHealthQA.csv"
]
dfs = []

for archivo in archivos:
    df_temp = pd.read_csv(archivo)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

In [ ]:
df

,Question,Answer,topic,split
0,What is (are) Non-Small Cell Lung Cancer ?,Key Points\n - Non-small ce...,cancer,train
1,Who is at risk for Non-Small Cell Lung Cancer? ?,Smoking is the major risk factor for non-small...,cancer,train
2,What are the symptoms of Non-Small Cell Lung C...,Signs of non-small cell lung cancer include a ...,cancer,test
3,How to diagnose Non-Small Cell Lung Cancer ?,Tests that examine the lungs are used to detec...,cancer,train
4,What is the outlook for Non-Small Cell Lung Ca...,Certain factors affect prognosis (chance of re...,cancer,train
...,...,...,...,...
8073,What is (are) Dry Mouth ?,Sjgren's Syndrome Clinic National Institute o...,SeniorHealth,test
8074,What is (are) Dry Mouth ?,For information about the clinical trial on th...,SeniorHealth,test
8075,What are the treatments for Dry Mouth ?,Dry mouth treatment will depend on what is cau...,SeniorHealth,train
8076,What is (are) Dry Mouth ?,You should avoid sticky and sugary foods. If y...,SeniorHealth,train


La columna "split" no sirve en este caso, por lo tanto, la eliminamos.

In [ ]:
df.drop(columns=["split"], inplace=True)

Tenemos el corpus unificado y con las columnas necesarias para las siguientes consultas. Por tanto, lo vamos a guardar en un archivo final .csv para cargarlo en nuestro proyecto.

Ahora, limpiemos las preguntas de cada documento para relacionarlos y observar los qrels más relevantes (mayor frecuencia).

In [ ]:
df["Question"] = df["Question"].str.replace("?", "")
df["Question"] = df["Question"].str.strip()

In [ ]:
df.to_csv("corpus.csv", index=False)

## 2. Explicación de las etiquetas o `qrels` en el dataframe

Las etiquetas para nuestra base de datos son las preguntas de la columna "Question". Los modelos solamente procesarán en la columna "Answer" y la pregunta será la consulta o el query que deberá recuperar los documentos que poseen la pregunta original.

In [ ]:
preguntas_duplicadas = df[df['Question'].duplicated(keep=False)]

In [ ]:
preguntas_duplicadas["Question"].nunique()

472

In [ ]:
preguntas_duplicadas['Question'].value_counts()

,count
Question,
What causes Causes of Diabetes,20
What is (are) High Blood Cholesterol,18
What is (are) Medicare and Continuing Care,14
What are the treatments for Breast Cancer,12
What is (are) Kidney Failure: Eat Right to Feel Right on Hemodialysis,12
...,...
What to do for Anemia of Inflammation and Chronic Disease,2
What is (are) Anemia of Inflammation and Chronic Disease,2
What is (are) Urinary Tract Infection In Adults,2


Las preguntas con solo 2 resultados no nos interesan, entonces solo tomaremos en cuenta las preguntas con resultados mayores a 8.

In [ ]:
cuentas = preguntas_duplicadas['Question'].value_counts()
preguntas_a_eliminar = cuentas[cuentas < 8].index

df_filtered = preguntas_duplicadas[~preguntas_duplicadas['Question'].isin(preguntas_a_eliminar)].copy()

In [ ]:
df_filtered['Question'].value_counts()

,count
Question,
What causes Causes of Diabetes,20
What is (are) High Blood Cholesterol,18
What is (are) Medicare and Continuing Care,14
What is (are) Kidney Failure: Eat Right to Feel Right on Hemodialysis,12
What are the treatments for Breast Cancer,12
What is (are) Skin Cancer,12
What is (are) Breast Cancer,11
What is (are) Colorectal Cancer,11
What are the treatments for Prostate Cancer,10


In [ ]:
preguntasClave = {
    "What causes Causes of Diabetes",
    "What is (are) High Blood Cholesterol",
    "What is (are) Medicare and Continuing Care",
    "What is (are) Kidney Failure: Eat Right to Feel Right on Hemodialysis",
    "What are the treatments for Breast Cancer",
    "What is (are) Skin Cancer",
    "What is (are) Breast Cancer",
    "What is (are) Colorectal Cancer",
    "What are the treatments for Prostate Cancer",
    "What is (are) Nutrition for Advanced Chronic Kidney Disease in Adults",
    "What is (are) Stroke",
    "Who is at risk for Prostate Cancer",
    "What is (are) Leukemia",
    "Who is at risk for Breast Cancer",
    "What is (are) Parkinson's Disease",
    "What is (are) Age-related Macular Degeneration",
    "What is (are) High Blood Pressure",
    "What is (are) Prostate Cancer"
}

In [ ]:
qrels_dict = {}

for pregunta in preguntasClave:
    # Filtrar el DataFrame para obtener los documentos que responden a esta pregunta específica
    indices_doc = df[df['Question'] == pregunta].index.tolist()
    qrels_dict[pregunta] = indices_doc

print("Diccionario de Qrels (Pregunta -> IDs de Documentos):")
for pregunta, ids in qrels_dict.items():
    print(f"\"{pregunta}\": {ids}")

Diccionario de Qrels (Pregunta -> IDs de Documentos):
"What is (are) Leukemia": [7813, 7819, 7820, 7821, 7822, 7823, 7829, 7830, 7831]
"What is (are) High Blood Cholesterol": [7962, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7975, 7976, 7977, 7978, 7979, 7980, 7983, 7984]
"What are the treatments for Breast Cancer": [664, 695, 7918, 7919, 7932, 7934, 7935, 7937, 7938, 7939, 7940, 7941]
"What is (are) High Blood Pressure": [8000, 8005, 8006, 8007, 8008, 8015, 8016, 8017, 8018]
"What is (are) Nutrition for Advanced Chronic Kidney Disease in Adults": [6257, 6258, 6259, 6260, 6261, 6262, 6263, 6264, 6265, 6267]
"What causes Causes of Diabetes": [6583, 6584, 6585, 6586, 6587, 6633, 6634, 6635, 6636, 6637, 7069, 7070, 7071, 7072, 7073, 7230, 7231, 7232, 7233, 7234]
"What is (are) Kidney Failure: Eat Right to Feel Right on Hemodialysis": [7087, 7088, 7089, 7090, 7091, 7092, 7093, 7094, 7095, 7096, 7097, 7098]
"What are the treatments for Prostate Cancer": [101, 7656, 7657, 7671, 76

# VI. Uso de los modelos

A continuación, el modelo binario siendo ejecutado con el corpus que genera sus propias matrices.

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
modeloBinario = ModeloBinario()
modeloBinario.ajustarCorpus(df['Answer'])

# Ejemplo de consulta
indicesResultados = modeloBinario.buscar("leukemia", k=5)
print("\nÍndices de documentos relevantes:", indicesResultados)

# Mostrar los documentos del DataFrame original
if len(indicesResultados) > 0:
    print("\nRespuestas encontradas:")
    print(df.iloc[indicesResultados])


Iniciando ajuste del Modelo Binario...
Ajuste completado. Documentos: 8078, Términos: 19519
Matriz de Ocurrencia (Documentos x Términos):
[[1 1 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]

Buscando: 'leukemia' con límite k=5
Documentos relevantes encontrados (total): 171
Documentos retornados (top k=5): 5
Top 5 resultados encontrados (ID):

Índices de documentos relevantes: [ 43  71 154 180 181]

Respuestas encontradas:
                                              Question  \
43   What is (are) Plasma Cell Neoplasms (Including...   
71           What are the treatments for Ewing Sarcoma   
154           Who is at risk for Merkel Cell Carcinoma   
180            What is (are) Myelodysplastic Syndromes   
181  What are the treatments for Myelodysplastic Sy...   

                                                Answer   topic  
43   Key Points\n                    - Plasma cell ...  cancer  
71   Key Points\n         

Después, tenemos al modelo vectorial TF-IDF que representa a los corpus por vectores. Se muestra un ranking.

In [ ]:
modeloVectorial = ModeloVectorialTfIdf()
modeloVectorial.ajustarCorpus(df['Answer'])

# Ejemplo de consulta con ranking (k=2)
resultadosRanking = modeloVectorial.buscar("leukemia", k=2)

if len(resultadosRanking) > 0:
    print("\nRespuestas encontradas (Top 2):")
    # Creamos un DataFrame para mostrar mejor los resultados
    dfResultados = pd.DataFrame(resultadosRanking, columns=['Documento ID', 'Similitud del Coseno'])

    # Respuestas reales del corpus
    dfResultados['Respuesta'] = dfResultados['Documento ID'].apply(lambda i: df.loc[i, 'Answer'])
    print(dfResultados[['Documento ID', 'Similitud del Coseno', 'Respuesta']])

Ajuste completado. Documentos: 8078, Términos: 19519
Muestra de la Matriz TF-IDF (Normalizada):
[[0.0216245  0.02207357 0.30867727 ... 0.         0.         0.        ]
 [0.         0.         0.03872171 ... 0.         0.         0.        ]
 [0.         0.         0.08231283 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.49190054]]
Top 2 resultados encontrados (ID, Similitud del Coseno):

Respuestas encontradas (Top 2):
   Documento ID  Similitud del Coseno  \
0          7827              0.744222   
1          7813              0.718835   

                                           Respuesta  
0  Treatment depends on a number of factors, incl...  
1  Leukemia is a cancer of the blood cells. It is...  


Terminamos con el modelo BM25 que utiliza probabilidades y una estructura similar y cambiada del TF-IDF.

In [ ]:
modeloBM25 = ModeloBM25(k1=1.2, b=0.75)
modeloBM25.ajustarCorpus(df['Answer'])

# Ejemplo de consulta con ranking (k=2)
resultadosRankingBM25 = modeloBM25.buscar("leukemia", k=2)

if len(resultadosRankingBM25) > 0:
    print("\nRespuestas encontradas (Top 2):")
    dfResultados = pd.DataFrame(resultadosRankingBM25, columns=['Documento ID', 'Puntuación BM25'])
    dfResultados['Respuesta'] = dfResultados['Documento ID'].apply(lambda i: df.loc[i, 'Answer'])
    print(dfResultados[['Documento ID', 'Puntuación BM25', 'Respuesta']])

Ajuste completado. Documentos: 8078, Términos: 19619
Longitud Promedio (avgdl): 232.33
Top 2 resultados encontrados (ID, Puntuación BM25):

Respuestas encontradas (Top 2):
   Documento ID  Puntuación BM25  \
0          7822         7.978854   
1          7813         7.813518   

                                           Respuesta  
0  There are four common types of leukemia. They ...  
1  Leukemia is a cancer of the blood cells. It is...  


# V. Almacenamiento de los modelos

Con pickle, podemos serializar los modelos de la siguiente manera. Definimos funciones para cargar y guardar los modelos anteriores entrenados.

In [ ]:
def guardarModelo(modelo, nombreArchivo):
    """ Guarda el objeto del modelo usando pickle. """
    try:
        with open(nombreArchivo, 'wb') as archivoSalida:
            pickle.dump(modelo, archivoSalida)
        print(f"\nModelo guardado exitosamente en: **{nombreArchivo}**")
    except Exception as e:
        print(f"Error al guardar el modelo: {e}")

def cargarModelo(nombreArchivo):
    """ Carga el objeto del modelo guardado usando pickle. """
    try:
        with open(nombreArchivo, 'rb') as archivoEntrada:
            modeloCargado = pickle.load(archivoEntrada)
        print(f"\nModelo cargado exitosamente desde: **{nombreArchivo}**")
        return modeloCargado
    except FileNotFoundError:
        print(f"\nError: Archivo **{nombreArchivo}** no encontrado.")
        return None
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return None

Y usamos la función para guardar los modelos.

In [ ]:
nombreArchivoModelo = 'modeloBinario.pkl'
# 1. Guardar el modelo binario
guardarModelo(modeloBinario, nombreArchivoModelo)


Modelo guardado exitosamente en: **modeloBinario.pkl**


In [ ]:
nombreArchivoModeloTfIdf = 'modeloTfIdf.pkl'
# 1. Guardar el modelo TF-IDF
guardarModelo(modeloVectorial, nombreArchivoModeloTfIdf)


Modelo guardado exitosamente en: **modeloTfIdf.pkl**


In [ ]:
nombreArchivoModeloBM25 = 'modeloBM25.pkl'
# 1. Guardar el modelo BM25
guardarModelo(modeloBM25, nombreArchivoModeloBM25)


Modelo guardado exitosamente en: **modeloBM25.pkl**


Podemos cargarlos usando el nombre asignado a cada archivo y la función definida anteriormente.

In [ ]:
modeloCargado = cargarModelo(nombreArchivoModelo)

if modeloCargado:
    consultaNueva = "heart"
    indicesResultadosCargados = modeloCargado.buscar(consultaNueva)

    if len(indicesResultadosCargados) > 0:
        print("\nRespuestas encontradas con Modelo Cargado:")
        print(df.iloc[indicesResultadosCargados])


Modelo cargado exitosamente desde: **modeloBinario.pkl**

Buscando: 'heart'
Documentos relevantes encontrados: 751

Respuestas encontradas con Modelo Cargado:
                                               Question  \
5     What are the stages of Non-Small Cell Lung Cancer   
13               What are the stages of Uterine Sarcoma   
25                    How to prevent Endometrial Cancer   
33            What are the stages of Endometrial Cancer   
43    What is (are) Plasma Cell Neoplasms (Including...   
...                                                 ...   
8058                         What is (are) Heart Attack   
8059           What are the treatments for Heart Attack   
8060                         What is (are) Heart Attack   
8061           What are the treatments for Heart Attack   
8062                         What is (are) Heart Attack   

                                                 Answer         topic  
5     Key Points\n                    - After lung c...    

# VI. Evaluación de los modelos

Se definen a continuación, funciones que calculan tres métricas importantes en la Recuperación de la Información.

In [ ]:
def calcularPrecisionK(documentosRecuperados: List[int], documentosRelevantes: List[int], k: int) -> float:
    recuperadosK = set(documentosRecuperados[:k])
    relevantes = set(documentosRelevantes)

    if not recuperadosK:
        return 0.0

    # Número de documentos relevantes recuperados entre los top K
    relevantesRecuperados = len(recuperadosK.intersection(relevantes))

    return relevantesRecuperados / k

def calcularRecallK(documentosRecuperados: List[int], documentosRelevantes: List[int], k: int) -> float:
    if not documentosRelevantes:
        return 0.0

    recuperadosK = set(documentosRecuperados[:k])
    relevantes = set(documentosRelevantes)

    # Número de documentos relevantes recuperados entre los top K
    relevantesRecuperados = len(recuperadosK.intersection(relevantes))

    # La exhaustividad se normaliza por el total de relevantes en el corpus
    return relevantesRecuperados / len(relevantes)

def calcularMAP(documentosRecuperados: List[int], documentosRelevantes: List[int]) -> float:
    precisiones = []
    numRelevantesEncontrados = 0

    relevantes = set(documentosRelevantes)

    for i, idDoc in enumerate(documentosRecuperados):
        if idDoc in relevantes:
            numRelevantesEncontrados += 1
            precisionEnK = numRelevantesEncontrados / (i + 1)
            precisiones.append(precisionEnK)

    # Si no hay documentos relevantes o no se recuperó ninguno, el MAP es 0
    if not documentosRelevantes or not precisiones:
        return 0.0

    # MAP es la suma de las precisiones en cada posición relevante, dividida
    # por el total de documentos relevantes para la consulta.
    return sum(precisiones) / len(documentosRelevantes)

Sabemos que los tres modelos tienen salidas distintas entre sí, debido a que los scores no son fácilmente calculables. Por ejemplo, el modelo binario no calcula scores, utiliza operaciones `OR` o `AND` para recuperar documentos. La siguiente función se encarga de devolver una lista de documentos para utilizar las métricas anteriores independientemente del modelo.

In [ ]:
def obtenerIDsRecuperados(resultadosModelo: List, nombreModelo: str) -> List[int]:
    if nombreModelo == 'ModeloBinario':
        # El Modelo Binario devuelve directamente una lista de IDs
        return [int(i) for i in resultadosModelo if isinstance(i, (int, np.integer, float))]

    elif isinstance(resultadosModelo, list) and len(resultadosModelo) > 0 and isinstance(resultadosModelo[0], (tuple, list)):
        # Modelos TF-IDF/BM25 devuelven una lista de tuplas (id_doc, score)
        return [int(item[0]) for item in resultadosModelo]

    return []

In [ ]:
K_EVALUACION = 10

modelos = {
    "Binario": modeloBinario,
    "TF-IDF": modeloVectorial,
    "BM25": modeloBM25
}

resultados_promedio = {
    "Modelo": [],
    f"P@{K_EVALUACION} Promedio": [],
    f"R@{K_EVALUACION} Promedio": [],
    "MAP Promedio": []
}

# Iteramos sobre cada modelo
for nombre_modelo, modelo in modelos.items():

    precisiones_k = []
    exhaustividades_k = []
    maps = []

    # Iteramos sobre cada pregunta Qrel
    for consulta_qrel, documentos_relevantes in qrels_dict.items():

        # 1. Obtener resultados del modelo
        # Pedimos K resultados al modelo
        resultados = modelo.buscar(consulta_qrel, k=K_EVALUACION)

        # 2. Extraer solo los IDs de los documentos recuperados
        id_recuperados = obtenerIDsRecuperados(resultados, nombre_modelo)

        # 3. Calcular métricas

        # Precisión @ K
        pk = calcularPrecisionK(id_recuperados, documentos_relevantes, k=K_EVALUACION)
        precisiones_k.append(pk)

        # Exhaustividad @ K
        rk = calcularRecallK(id_recuperados, documentos_relevantes, k=K_EVALUACION)
        exhaustividades_k.append(rk)

        # MAP
        # Para MAP, el cálculo se hace sobre el ranking completo (por eso no pasamos K aquí),
        # aunque si tu modelo.buscar solo devuelve K resultados, el MAP estará limitado por K.
        map_score = calcularMAP(id_recuperados, documentos_relevantes)
        maps.append(map_score)

        print(f"  Query: '{consulta_qrel[:40]}...' | P@{K_EVALUACION}={pk:.3f} | R@{K_EVALUACION}={rk:.3f} | MAP={map_score:.3f}")

    # 4. Calcular los promedios (Mean Average Precision, Mean Recall, Mean P)
    promedio_pk = np.mean(precisiones_k)
    promedio_rk = np.mean(exhaustividades_k)
    promedio_map = np.mean(maps)

    print(f"\nResultados Promedio para {nombre_modelo}:")
    print(f"  P@{K_EVALUACION} Promedio: {promedio_pk:.4f}")
    print(f"  R@{K_EVALUACION} Promedio: {promedio_rk:.4f}")
    print(f"  MAP Promedio: {promedio_map:.4f}")

    # 5. Almacenar los resultados para el reporte final
    resultados_promedio["Modelo"].append(nombre_modelo)
    resultados_promedio[f"P@{K_EVALUACION} Promedio"].append(promedio_pk)
    resultados_promedio[f"R@{K_EVALUACION} Promedio"].append(promedio_rk)
    resultados_promedio["MAP Promedio"].append(promedio_map)


Buscando: 'What is (are) Leukemia' con límite k=10
Documentos relevantes encontrados (total): 171
Documentos retornados (top k=10): 10
Top 10 resultados encontrados (ID):
  Query: 'What is (are) Leukemia...' | P@10=0.000 | R@10=0.000 | MAP=0.000

Buscando: 'What is (are) High Blood Cholesterol' con límite k=10
Documentos relevantes encontrados (total): 91
Documentos retornados (top k=10): 10
Top 10 resultados encontrados (ID):
  Query: 'What is (are) High Blood Cholesterol...' | P@10=0.000 | R@10=0.000 | MAP=0.000

Buscando: 'What are the treatments for Breast Cancer' con límite k=10
Documentos relevantes encontrados (total): 21
Documentos retornados (top k=10): 10
Top 10 resultados encontrados (ID):
  Query: 'What are the treatments for Breast Cance...' | P@10=0.000 | R@10=0.000 | MAP=0.000

Buscando: 'What is (are) High Blood Pressure' con límite k=10
Documentos relevantes encontrados (total): 276
Documentos retornados (top k=10): 10
Top 10 resultados encontrados (ID):
  Query: 'Wha

Todo se visualizó desordenado, así que usamos un dataframe para imprimir los resultados de los tres modelos.

In [ ]:
df_reporte = pd.DataFrame(resultados_promedio)
print("\n" + "="*50)
print(f"REPORTE FINAL DE EVALUACIÓN (K={K_EVALUACION})")
print("="*50)
print(df_reporte.set_index("Modelo").round(4))


REPORTE FINAL DE EVALUACIÓN (K=10)
         P@10 Promedio  R@10 Promedio  MAP Promedio
Modelo                                             
Binario         0.0000         0.0000        0.0000
TF-IDF          0.2944         0.2689        0.1801
BM25            0.3444         0.3338        0.2220


En cuestión, el modelo binario no logró recuperar documentos, debido a la complejidad de las preguntas y el uso de sus operaciones para extraer documentos.

El modelo TF-IDF logró hacerlo mejor, pero BM25 tiene una mayor puntuación comparándose entre los tres.